<a href="https://colab.research.google.com/github/ZoliN/colab/blob/main/blade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/MyDrive/Colab Notebooks/zip.zip" -d /content/
%cd /content/zip
!mkdir build
%cd build
!cmake .. -DCMAKE_BUILD_TYPE=Release

In [ ]:
%cd /content
!wget  http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip
!mkdir pics
!unzip DIV2K_train_HR.zip -d pics/

In [ ]:
%cd /content/pics/DIV2K_train_HR
!sh /content/drive/MyDrive/blade/copypics.sh

/content/pics/DIV2K_train_HR


In [ ]:
 %cd /content/zip/build
!make -j4

In [ ]:
!cp "/content/drive/MyDrive/Colab Notebooks/2023_5_28_1_35_27.filter"  /content/zip/filters/

In [ ]:
!pwd

In [ ]:
%cd /content/zip/train_images
!mv *.png ./p/
!mv ./p/08.png .
!for i in {1..1000}; do cp 08.png "test$i.png"; done

/content/zip/train_images


In [ ]:
%cd /content/zip/
!./build/blade
!cp -r filters /content/drive/MyDrive/blade/
!cp -r result_images /content/drive/MyDrive/blade/
!cp temp_images/*.csv /content/drive/MyDrive/blade/
while True:pass

In [ ]:
!zip -r /content/drive/MyDrive/bladearch/src.zip /content/drive/MyDrive/blade/*

In [ ]:
%cd /content/
!mkdir build
%cd build
!cmake /content/drive/MyDrive/superrestf/bladesr -DCMAKE_BUILD_TYPE=Release
!pip install python_magic
!apt-get install libmagic1

In [ ]:
 %cd /content/build
!make -j4

In [ ]:
import os
os.makedirs('/content/anal_images', exist_ok=True)
!/content/build/blade

import os
import tensorflow as tf
import cv2

def iterate_files(directory):
    for file in os.listdir(directory):
        yield os.path.join(directory, file)

inpath = '/content/anal_images'
outpath = '/content/anal_tensors'
os.makedirs(outpath, exist_ok=True)
# Iterate over the files of the current directory
for file in iterate_files(inpath):
    filename = os.path.basename(file)
    print(filename)
    im_input = cv2.imread(file, -1)
    print(im_input.shape)

    byte_string = tf.io.serialize_tensor(im_input)
    # Write the byte string to a file
    with tf.io.gfile.GFile(outpath + '/' + filename[:-5] + '.bin', 'wb') as f:
        f.write(byte_string.numpy())


In [7]:
import os
import tensorflow as tf
import cv2
import numpy as np
from importlib import reload

scale = 2
scalef = float(scale)
invscale = 1./scalef

img_raw = tf.io.read_file('/content/drive/MyDrive/superrestf/bladesr/test_images/12.png')
im_input = tf.image.decode_image(img_raw)
im_input = tf.cast(im_input, dtype=tf.float32)/255.0



# img_raw = tf.io.read_file('/content/pic12.png')
# im_input2 = tf.image.decode_image(img_raw, channels=1)
# im_input2 = tf.cast(im_input2, dtype=tf.float32)/255.0
# print('PSNR: ' + str(tf.image.psnr(im_input2[8:512-8,8:512-8,:], im_input[8:512-8,8:512-8,:], max_val=1.0).numpy()))

orig_sz = tf.shape(im_input)[0:2]
lr_sz = orig_sz // scale
full_sz = lr_sz * scale


analpath = '/content/anal_tensors'
byte_string = []
with tf.io.gfile.GFile(analpath + '/' + '12.png' + '.bin', 'rb') as f:
    byte_string = f.read()

# Deserialize the tensor from the byte string
anal_tensor = tf.io.parse_tensor(byte_string, out_type=tf.float32)
print(tf.shape(input=anal_tensor))

lam1, lam2, c, s = tf.unstack(anal_tensor, axis=-1)

Dtr = 3000. * 1.525902E-05 * 0.860704768
Dth = 1000. * 1.525902E-05 * 0.860704768 * 1. / Dtr;
kDetail = 0.5;
kDenoise = 1.55;
kStretch = 4.;
kShrink = 1.;

Dtr=0.0039259493
Dth=0.33213648
kDetail=0.6002224
kDenoise=0.5469237
kStretch=2.291745
kShrink=2.3721561

Dtr=0.48
Dth=0.801
kDetail=0.1388774
kDenoise=0.272592
kStretch=3.6042476
kShrink=0.590034



A = (lam1 - lam2) / (lam1 + lam2 + 0.0000000001);
A = tf.clip_by_value(A, 0.1, 1.0);

D = 1. - tf.sqrt(lam1) / Dtr + Dth
D = tf.clip_by_value(D, 0.0, 1.0)

k1h = kDetail * (kStretch * A +  (1. - A));
k2h = kDetail / (kShrink * A +  (1. - A));

k1 = ((1.0 - D)*k1h + D*kDenoise);
k2 = ((1.0 - D)*k2h + D*kDenoise);
k1 *= k1;
k2 *= k2;

x2 = c;
y2 = s;
x1 = s;
y1 = -c;

b11 = k1*x1*x1 + x2*x2*k2;
b12 = k1*x1*y1 + x2*y2*k2;
b22 = k1*y1*y1 + y2*y2*k2;

det = b11*b22 - b12*b12 + 0.0000000001;

m11 = b22 / det;
m22 = b11 / det;
m12 = -b12 / det;

kerp = tf.stack((m11, m22, m12), axis = -1)
kerp = tf.image.resize(kerp, full_sz, method='bilinear')
m11, m22, m12 = tf.split(kerp, 3, axis = -1)


######## k2
# k1h = kDetail2 * (kStretch * A +  (1. - A));
# k2h = kDetail2 / (kShrink * A +  (1. - A));

# k1 = ((1.0 - D)*k1h + D*kDenoise2);
# k2 = ((1.0 - D)*k2h + D*kDenoise2);
# k1 *= k1;
# k2 *= k2;

# x2 = c;
# y2 = s;
# x1 = s;
# y1 = -c;

# b11 = k1*x1*x1 + x2*x2*k2;
# b12 = k1*x1*y1 + x2*y2*k2;
# b22 = k1*y1*y1 + y2*y2*k2;

# det = b11*b22 - b12*b12 + 0.0000000001;

# m11_2 = b22 / det;
# m22_2 = b11 / det;
# m12_2 = -b12 / det;

# kerp = tf.stack((m11_2, m22_2, m12_2), axis = -1)
# kerp = tf.image.resize(kerp, full_sz, method='bilinear')
# m11_2, m22_2, m12_2 = tf.split(kerp, 3, axis = -1)


im_lr = tf.image.resize(im_input, lr_sz, method='area')

im_hr = tf.image.resize(im_lr, full_sz, method='nearest')

cv2.imwrite('/content/im_hr.png', im_hr.numpy()*255.0)

im_hr2 = tf.image.resize(im_lr, full_sz, method='lanczos3')
cv2.imwrite('/content/im_hr2.png', im_hr2.numpy()*255.0)


paddings = tf.constant([[2 * scale, 2 * scale], [2 * scale, 2 * scale], [0, 0]])
# paddings = tf.constant([[2, 2], [2, 2], [0, 0]])
im_padded = tf.pad(im_hr, paddings, "SYMMETRIC")

#tf.convert_to_tensor(value, dtype=None
im_dst = tf.fill(tf.concat( full_sz, tf.shape(im_lr)[2]), 0.0)

im_dst = tf.fill(tf.shape(im_hr), 0.0)
im_w = tf.fill(tf.shape(im_hr), 0.0)


print(tf.shape(input=im_dst))
print(tf.shape(input=im_padded))


# Create two 1D arrays
x_coords = np.arange(float(full_sz[0]))
y_coords = np.arange(float(full_sz[1]))

# Create a 2D NumPy array containing x and y indices
indices = np.meshgrid(x_coords, y_coords)

srcpos1 = np.stack([indices[0], indices[1]], 2)
srcpos1 = (srcpos1 + 0.4995) * invscale
srcpos = tf.convert_to_tensor(srcpos1, dtype=tf.float32)
srcMidPos = tf.floor(srcpos) + 0.5;

ls = 3.
pi = 3.14159265
for y in range(5):
  ys = y * scale
  y1 = float(y - 2)
  for x in range(5):
    xs = x * scale
    x1 = float(x - 2)
    posk = (srcMidPos + (x1, y1) - srcpos) * scalef
    poskx, posky = tf.split(posk, 2, axis = -1)
    #if (x==2 and y==2):
    #  print(poskx.numpy()[0:4,0:4])
    # kwx = tf.where(poskx == 0.0, 1.0, tf.where(poskx > ls, 0.0,  ls * tf.sin(pi * poskx) * tf.sin(pi * poskx / ls) / (pi * pi * poskx * poskx)))
    # kwy = tf.where(posky == 0.0, 1.0, tf.where(posky > ls, 0.0,  ls * tf.sin(pi * posky) * tf.sin(pi * posky / ls) / (pi * pi * posky * posky)))
    # kw = kwx * kwy

    kw = tf.exp(-0.5 * ((poskx*m11+posky*m12)*poskx+(poskx*m12+posky*m22)*posky) )
    # kw2 = tf.exp(-0.5 * ((poskx*m11_2+posky*m12_2)*poskx+(poskx*m12_2+posky*m22_2)*posky) )
    # kw = kw1 * 2.0 - kw2 * 1.0
    if (x==4 and y==2):
      print(kw.numpy()[0:4,0:4])
    im_w += kw
    im_dst+= kw * im_padded[ys:ys+full_sz[0], xs: xs+full_sz[1]]

halfRatio = invscale * 0.5

# for y in range(5):
#   y1 = float(y - 2)
#   for x in range(5):
#     x1 = float(x - 2)
#     srcposc = srcpos + (x1 * invscale, y1 * invscale)
#     comp = tf.where(tf.abs(srcposc - tf.floor(srcposc) - 0.5) < halfRatio, 1.0, 0.0)
#     compx, compy = tf.split(comp, 2, axis = -1)
#     kw = tf.where(compx * compx == 1.0, tf.exp(-0.5 * ((x1*m11+y1*m12)*x1+(x1*m12+y1*m22)*y1) ), 0.0)
#     #if (x==3 and y==2):
#     #  print(kw.numpy()[200:204,200:204])
#     im_w += kw
#     im_dst+= kw * im_padded[y:y+full_sz[0], x: x+full_sz[1]]


im_dst = im_dst / (im_w + 0.0000001)

cv2.imwrite('/content/out.png', im_dst.numpy()*255.0)

cv2.imwrite('/content/test.jpg', A.numpy()*255.0)

print(tf.shape(input=D))
print('PSNR: ' + str(tf.image.psnr(im_dst, im_input, max_val=1.0).numpy()))
print('PSNR upsampl: ' + str(tf.image.psnr(im_hr2, im_input, max_val=1.0).numpy()))



checkpoint_filepath = '/content/checkpoint'

model = models.SRBlade(params)
try:
  model.load_weights(checkpoint_filepath)
except:
  print("Couldn't load weights from ",checkpoint_filepath)

model.params['net_input_size'] = tf.Variable(initial_value = 512, trainable=False, name= 'params_net_input_size')

model.compile(loss=['mse', returnPrediction], metrics={'output_1':PSNR},
  run_eagerly=True)

outimg = model( {'image_input' : tf.expand_dims(im_lr,0), 'anal_input' : tf.expand_dims(anal_tensor,0)})

print('PSNR mmodel: ' + str(tf.image.psnr(tf.squeeze(outimg,0), im_input, max_val=1.0).numpy()))
cv2.imwrite('/content/outm.png', tf.squeeze(outimg,0).numpy()*255.0)



PSNR: 30.554575


NotFoundError: /content/anal_tensors/12.png.bin; No such file or directory

In [5]:
#!ln -sf /content/drive/MyDrive/superrestf/bladesr/test_images /content/input
!ln -sf /content/pics/DIV2K_train_HR /content/input
!cp /content/drive/MyDrive/superrestf/bladesr/test_images/* /content/input/
#!rm /content/checkpoint*
%cd /content/drive/MyDrive/
import tensorflow as tf
import numpy as np
import os
import sys
import cv2
import skimage
import skimage.io
import logging
from keras.models import load_model
from tensorflow.python.ops.math_ops import truediv
import tensorflow_probability as tfp
#import tensorflow_addons as tfa


import superrestf.models2 as models
import superrestf.data_pipeline as dp

from importlib import reload

reload(models)
reload(dp)


numImgChs = 1

logging.basicConfig(format="[%(process)d] %(levelname)s %(filename)s:%(lineno)s | %(message)s")
log = logging.getLogger("train")
log.setLevel(logging.INFO)

def PSNR(im1, im2):
  return tf.image.psnr(im1, im2, max_val=1.0)

def returnPrediction(target, pred):
  return pred

def return0(target, pred):
  return 0


def l2_lossHist(target, prediction):
  loss = tf.reduce_mean(input_tensor=tf.square(target-prediction))

  percs = np.linspace(1, 100, 64, endpoint=False, dtype=np.float32)
  hist1 = tfp.stats.percentile(target, percs, [1,2,3])
  hist2 = tfp.stats.percentile(prediction, percs, [1,2,3])
  histdiff2 = tf.reduce_mean(input_tensor=tf.square(hist1-hist2))
  return loss + histdiff2 * 10.

def PSNR_w(target, prediction):
  rgb, segmented = tf.split(target, [numImgChs,1], 3)
  squares = segmented * tf.square(rgb-prediction)
  squares = tf.reshape(squares, [tf.shape(input=squares)[0], -1])
  p = (-10/np.log(10))*tf.math.log(tf.reduce_sum(input_tensor=squares, axis=[1])
          / tf.reduce_sum(input_tensor=segmented, axis=[1,2,3]))
  return p


def l2_w(target, prediction):
  rgb, segmented = tf.split(target, [numImgChs,1], 3)
  squares = segmented * tf.square(rgb-prediction)
  l2 = tf.reduce_sum(input_tensor=squares) / tf.reduce_sum(input_tensor=segmented)
  return l2

params = {}
params['net_input_size'] = 256

checkpoint_filepath = '/content/checkpoint'
data_dir = '/content/filelist.txt'
validation_dir = '/content/filelistv.txt'
#validation_dir = ''
output_resolution = [378, 504]
output_resolution = [384, 510]
# output_resolution = [1512, 2016]
fliplr=True
flipud=True
rotate=False
random_crop=True
shuffle=False
batch_size = 128
learning_rate = 1e-3

net_input_size = params['net_input_size']

train_samples = dp.ImageFilesDataPipeline(
    data_dir,
    shuffle=shuffle,
    net_input_size=params['net_input_size'],
    capacity=1,
    min_after_dequeue = 4,
    batch_size=batch_size,
    fliplr=fliplr, flipud=flipud, rotate=rotate,
    random_crop=random_crop,
    custaugment=False,
    exposuremult=False,
    output_resolution=output_resolution,
    keras = True,
    repeat = True).samples

validation_samples = None
if validation_dir != '':
  validation_samples = dp.ImageFilesDataPipeline(
      validation_dir,
      net_input_size=params['net_input_size'],
      capacity=1,
      batch_size=1,
      fliplr=False,flipud=False, rotate=False,
      random_crop=True,
      exposuremult=False,
      output_resolution=output_resolution,
      keras = True,
      repeat = False).samples


opti=tf.keras.optimizers.Adam(learning_rate, use_ema=False, ema_momentum=0.1, weight_decay=0.001)
#opti=tfa.optimizers.MovingAverage(opti, 0.99)
model = models.SRBlade(params)
#model = models.HDRNet(params)

#model.run_eagerly = False
#model = load_model('/content/hdrnet/train/out', compile=False)
try:
  model.load_weights(checkpoint_filepath)
except:
  print("Couldn't load weights from ",checkpoint_filepath)

model.compile(optimizer=opti, loss=['mse', returnPrediction], metrics={'output_1':PSNR},
  run_eagerly=False)
# model.compile(optimizer=opti, loss=[l2_w, returnPrediction], metrics={'output_1':PSNR_w, 'output_2':return0},
# run_eagerly=False)


model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
#model_checkpoint_callback = tfa.callbacks.AverageModelCheckpoint(
  update_weights=True,
  filepath=checkpoint_filepath,
  save_weights_only=True,
  monitor= ('val_' if validation_dir != '' else '') + 'PSNR',
  #monitor= 'output_1_PSNR',
  mode='max',
  verbose=1,
  save_freq = 'epoch',
  save_best_only=True)



history = model.fit(train_samples, epochs=50, steps_per_epoch = 200//batch_size, callbacks=[model_checkpoint_callback],
  validation_data = validation_samples, validation_freq = 1, verbose = 'auto')




cp: -r not specified; omitting directory '/content/drive/MyDrive/superrestf/bladesr/test_images/t'
cp: -r not specified; omitting directory '/content/drive/MyDrive/superrestf/bladesr/test_images/t2'
cp: -r not specified; omitting directory '/content/drive/MyDrive/superrestf/bladesr/test_images/test_images'
/content/drive/MyDrive
Tensor("strided_slice_1:0", shape=(), dtype=string)
Tensor("strided_slice_2:0", shape=(), dtype=string)
Tensor("Shape:0", shape=(3,), dtype=int32)
Tensor("Shape_1:0", shape=(3,), dtype=int32)
Tensor("strided_slice_1:0", shape=(), dtype=string)
Tensor("strided_slice_2:0", shape=(), dtype=string)
Tensor("Shape:0", shape=(3,), dtype=int32)
Tensor("Shape_1:0", shape=(3,), dtype=int32)
Epoch 1/50
Tensor("sr_blade_4/Shape_1:0", shape=(4,), dtype=int32, device=/device:GPU:0)
Tensor("sr_blade_4/mul:0", shape=(2,), dtype=int32, device=/device:GPU:0)
Tensor("sr_blade_4/Shape_2:0", shape=(4,), dtype=int32, device=/device:GPU:0)
Tensor("sr_blade_4/mul:0", shape=(2,), dtype

Tensor("sr_blade_4/Shape_1:0", shape=(4,), dtype=int32, device=/device:GPU:0)
Tensor("sr_blade_4/mul:0", shape=(2,), dtype=int32, device=/device:GPU:0)
Tensor("sr_blade_4/Shape_2:0", shape=(4,), dtype=int32, device=/device:GPU:0)
Tensor("sr_blade_4/mul:0", shape=(2,), dtype=int32, device=/device:GPU:0)
Tensor("sr_blade_4/Shape_3:0", shape=(4,), dtype=int32, device=/device:GPU:0)
Tensor("sr_blade_4/mul:0", shape=(2,), dtype=int32, device=/device:GPU:0)
Tensor("sr_blade_4/Shape_4:0", shape=(4,), dtype=int32, device=/device:GPU:0)
Tensor("sr_blade_4/mul:0", shape=(2,), dtype=int32, device=/device:GPU:0)


1/1 [==============================] - ETA: 0s - loss: 0.0012 - PSNR: 34.2573Tensor("sr_blade_4/Shape_1:0", shape=(4,), dtype=int32, device=/device:GPU:0)
Tensor("sr_blade_4/mul:0", shape=(2,), dtype=int32, device=/device:GPU:0)
Tensor("sr_blade_4/Shape_2:0", shape=(4,), dtype=int32, device=/device:GPU:0)
Tensor("sr_blade_4/mul:0", shape=(2,), dtype=int32, device=/device:GPU:0)
Tensor("sr_blade_4/Shape_3:0", shape=(4,), dtype=int32, device=/device:GPU:0)
Tensor("sr_blade_4/mul:0", shape=(2,), dtype=int32, device=/device:GPU:0)
Tensor("sr_blade_4/Shape_4:0", shape=(4,), dtype=int32, device=/device:GPU:0)
Tensor("sr_blade_4/mul:0", shape=(2,), dtype=int32, device=/device:GPU:0)

Epoch 1: val_PSNR improved from -inf to 31.42937, saving model to /content/checkpoint
1/1 [==============================] - 51s 51s/step - loss: 0.0012 - PSNR: 34.2573 - val_loss: 7.1955e-04 - val_PSNR: 31.4294
Epoch 2/50
1/1 [==============================] - ETA: 0s - loss: 0.0013 - PSNR: 34.0497
Epoch 2: val_

KeyboardInterrupt: 

In [7]:

print(model.Dtr)
print(model.Dth)

print(model.kDetail)
print(model.kDenoise)
print(model.kStretch)
print(model.kShrink)

print(model.kDetail2)
print(model.kDenoise2)
print(model.kStretch2)
print(model.kShrink2)

print(3000. * 1.525902E-05 * 0.860704768)
print(100. * 1.525902E-05 * 0.860704768 / (3000. * 1.525902E-05 * 0.860704768))

<tf.Variable 'sr_blade_5/Dtr:0' shape=() dtype=float32, numpy=0.039400533>
<tf.Variable 'sr_blade_5/Dth:0' shape=() dtype=float32, numpy=0.33333334>
<tf.Variable 'sr_blade_5/kDetail:0' shape=() dtype=float32, numpy=0.27>
<tf.Variable 'sr_blade_5/kDenoise:0' shape=() dtype=float32, numpy=0.55>
<tf.Variable 'sr_blade_5/kStretch:0' shape=() dtype=float32, numpy=4.0>
<tf.Variable 'sr_blade_5/kShrink:0' shape=() dtype=float32, numpy=1.0>
<tf.Variable 'sr_blade_5/kDetail2:0' shape=() dtype=float32, numpy=0.54>
<tf.Variable 'sr_blade_5/kDenoise2:0' shape=() dtype=float32, numpy=1.1>
<tf.Variable 'sr_blade_5/kStretch2:0' shape=() dtype=float32, numpy=4.0>
<tf.Variable 'sr_blade_5/kShrink2:0' shape=() dtype=float32, numpy=1.0>
0.039400533807022076
0.03333333333333333


In [7]:
x_coords = np.arange(4)
y_coords = np.arange(4)

# Create a 2D NumPy array containing x and y indices
indices = np.meshgrid(x_coords, y_coords)

indices[0] = indices[0] % 2
indices[1] = indices[1] % 2
indices = indices[1] * 2 + indices[0]
print(indices)

[[0 1 0 1]
 [2 3 2 3]
 [0 1 0 1]
 [2 3 2 3]]


In [5]:
!ln -s /content/drive/MyDrive/superrestf/bladesr/test_images /content/input
%cd /content/drive/MyDrive/superrestf/bin
!python train2.py

^C
